In [1]:
import sys
sys.path.append("D:/Yuichi/MemoryAge_Transformer_xLARGE")
# the code needs to search for the modules in the parent directory

from torcheeg import transforms
from torcheeg.datasets import MemoryAgeDataset
from torcheeg.model_selection import KFoldCrossSubject
from torcheeg.models import LFP_Transformer_LARGE
from torcheeg.trainers import ClassifierTrainer

import logging
import os
import random
import time
import os
import multiprocessing
CPU_num = multiprocessing.cpu_count()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch.utils.data.dataloader import DataLoader

#### Set the random number seed in all modules to guarantee the same result when running again.

In [2]:
def seed_everything(seed):
    
    # Sets the random seed for Python's built-in random library. This will affect random operations like random.shuffle or random.randint.
    random.seed(seed) 

    #Sets the random seed for NumPy. This will affect random operations in NumPy like np.random.randn or np.random.shuffle.
    np.random.seed(seed)

    # Sets the random seed for hash functions in Python. The hash functions are used in Python dictionaries and sets,
    # and this ensures that they behave the same way each time you run the program.
    os.environ["PYTHONHASHSEED"] = str(seed)

    # Sets the random seed for PyTorch's CPU random number generator. This will affect random operations in PyTorch like random tensor generation.
    torch.manual_seed(seed)

    # Sets the random seed for PyTorch's CUDA random number generator, which affects random number generation on the GPU.
    torch.cuda.manual_seed(seed)

    # Ensures that every time you run your code, the algorithms used by cuDNN produce deterministic results.
    # NOTE: By default, cuDNN uses algorithms that are optimized for performance, but these algorithms might use approximations or other shortcuts 
    # that can result in small differences in output. This is known as being "non-deterministic."
    torch.backends.cudnn.deterministic = True  

    # Disables the cudNN auto-tuner, which optimizes cuDNN to find the best algorithm to use for your hardware. 
    # By disabling this, you make sure that cuDNN uses a deterministic algorithm.
    # NOTE: The cuDNN library provides several algorithms to perform operations commonly used in deep learning, like convolutions and pooling.
    # These algorithms can vary in their internal implementations and may have different performance and memory characteristics. 
    # The cuDNN auto-tuner is designed to automatically select the most efficient algorithm for your specific hardware configuration. 
    torch.backends.cudnn.benchmark = False

random_state = 19890516
seed_everything(random_state)

In [3]:
def load_val_batches(val_loader, num_batches=5):
    '''
    This function is designed to load a small number of batches from the validation set.
    I will load validation set in different runs, and compare whether the samples are in the same order.
    '''
    batches = []
    for i, batch in enumerate(val_loader):
        if i >= num_batches:
            break
        batches.append(batch)
    return batches

# Building Deep Learning Pipelines Using TorchEEG

#### Step 1: Initialize the Dataset

In [4]:
dataset = MemoryAgeDataset(io_path='./processed_dataset/',  
                           root_path='./raw_dataset/data_memory_age',
                           online_transform=transforms.ToTensor(),     # used in __getitem__()
                           num_worker=1,
                           io_mode='pickle')

dataset does not exist at path ./processed_dataset/, generating files to path...


[PROCESS]: 100%|██████████| 24/24 [04:39<00:00, 11.66s/it]

dataset already exists at path ./processed_dataset/, reading from path...


In [5]:
dataset.info

,subject_id,clip_id,label,trial_id
0,ym212,ym212_Recent_0,0,0
1,ym212,ym212_Recent_1,0,0
2,ym212,ym212_Recent_2,0,0
3,ym212,ym212_Recent_3,0,0
4,ym212,ym212_Recent_4,0,0
...,...,...,...,...
43908,ym227,ym227_Remote_1363,1,1
43909,ym227,ym227_Remote_1364,1,1
43910,ym227,ym227_Remote_1365,1,1
43911,ym227,ym227_Remote_1366,1,1


#### update: 20240126, shuffle the label column

In [ ]:
np.random.seed(42) 

# Shuffle only the 'label' column
shuffled_labels = dataset['label'].sample(frac=1).reset_index(drop=True)
# frac=1 means to return all rows, so none are dropped in the shuffle.

# Assign the shuffled labels back to the DataFrame
dataset['label'] = shuffled_labels


#### Step 2: Divide the Training and Test samples in the Dataset

In [6]:
k_fold = KFoldCrossSubject(n_splits=12, shuffle=False, split_path='./processed_dataset/cross_subject_k_fold_spilt')
# shulffle=False, so the order of the subjects in test is ym212, ym213, ..., ym227
# n_spilts=the num of subjects, so leave-one-subject-out cross validation

#### Step 3: Define the Model and Start Training

In [7]:
best_f1_scores_all = []

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    
    # ------------------------------------------- 
    # Initialize the model
    model = LFP_Transformer_LARGE(depth=5)  # 5 transformer blocks, namely, x LARGE MODEL
    
    # ------------------------------------------- 
    # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
    # The major parameters of the trainer are defined in ClassifierTrainer
    trainer = ClassifierTrainer(model=model,
                                lr=1e-4,
                                devices=1,
                                accelerator='gpu',
                                num_classes=2,
                                weight_decay=1e-4,
                                save_attention=True,
                                save_test_f1=True)

    # ------------------------------------------- 
    # Initialize several batches of training samples and test samples
    # Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset.
    train_loader = DataLoader(train_dataset,
                              batch_size=128,
                              shuffle=True,
                              num_workers=CPU_num)
    
    val_loader = DataLoader(val_dataset,
                            batch_size=128,
                            shuffle=False,
                            num_workers=CPU_num)
    # shuffle=False, very important, do not shuffle the samples in the validation set
    # in the validation and test phase, just feed all the samples in the default order,
    # for example, in fold0, the order is ym212_Recent_0, ym212_Recent_1, ym212_Recent_2, ...

    # ------------------------------------------- 
    # Do 50 rounds of training
    trainer.fit(train_loader, val_loader, fold_idx=i, max_epochs=30)

    # ------------------------------------------- 
    # Test the model on the test set
    # during the test period, for each batch with batch_size samples, the desired attention scores, 
    # i.e., the attention scores between the cls token and the patch tokens in the first transformer block,
    # are saved in the trainer.saved_attention, which is a list with several arrays (shape = (batch_size, num_heads, num_patches))
    trainer.test(val_loader, fold_idx=i)

    # ----- Process the predictions of the test samples, attach each prediction to the corresponding sample
    saved_predictions = trainer.saved_predictions_all
    saved_predictions = np.concatenate(saved_predictions, axis=0)  # shape = (num_samples, num_classes) = (num_samples, 2)
    # apply softmax to each row, namely, the predictions for each sample
    saved_predictions = np.exp(saved_predictions) / np.sum(np.exp(saved_predictions), axis=1, keepdims=True)
    # to pandas dataframe. the first column is 'class 0 probability', the second column is 'class 1 probability'
    saved_predictions = pd.DataFrame(saved_predictions, columns=['class 0 probability', 'class 1 probability'])
    # load the corresponding test samples
    test_sample_info = pd.read_csv(f'./processed_dataset/cross_subject_k_fold_spilt/test_fold_{i}.csv')
    # concatenate the test sample info and the predictions
    test_sample_info = pd.concat([test_sample_info, saved_predictions], axis=1)
    # save the test sample info and the predictions
    test_sample_info.to_csv(f'./processed_dataset/cross_subject_k_fold_spilt/test_fold_{i}.csv', index=False)

    # ----- Process the attention scores
    # Access the saved attention matrices in the TEST phase
    attention_matrices = trainer.saved_attention_all
    # concat attention_matrices in the batch dimension
    attention_matrices = np.concatenate(attention_matrices, axis=0)  # shape = (num_samples, num_heads, num_patches) = (num_samples, 4, 17)
    # create a folder in ./processed_dataset/ to save the attention matrices if not exist
    if not os.path.exists('./processed_dataset/attention_scores/'):
        os.makedirs('./processed_dataset/attention_scores/')
    np.save('./processed_dataset/attention_scores/attention_from_the_test_subject_in_Fold%d.npy' % i, attention_matrices)

    # ----- save the base test F1 scores for each fold (mouse) -----
    best_f1_scores_all.append(trainer.test_f1.cpu().numpy())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


-------------------- FOLD 0 -------------------
train_subjects: ['ym223', 'ym215', 'ym214', 'ym227', 'ym213', 'ym226', 'ym220', 'ym224', 'ym218', 'ym219', 'ym222']
test_subjects: ['ym212']
------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                  | Params
--------------------------------------------------------
0 | model         | LFP_Transformer_LARGE | 842 K 
1 | _ce_fn        | CrossEntropyLoss      | 0     
2 | train_loss    | MeanMetric            | 0     
3 | val_loss      | MeanMetric            | 0     
4 | test_loss     | MeanMetric            | 0     
5 | train_metrics | MetricCollection      | 0     
6 | val_metrics   | MetricCollection      | 0     
7 | test_metrics  | MetricCollection      | 0     
--------------------------------------------------------
842 K     Trainable params
0         Non-trainable params
842 K     Total params
3.371     Total estimated model params size (MB)



Epoch 0: [Val] val_loss: 0.756  val_f1score: 0.008  val_accuracy: 0.008  

Metric val_loss improved. New best score: 0.445


...

Epoch 4: [Val] val_loss: 0.467  val_f1score: 0.773  val_accuracy: 0.774     [Train] train_loss: 0.288  train_f1score: 0.873  train_accuracy: 0.873  
Epoch 5: [Val] val_loss: 0.581  val_f1score: 0.756  val_accuracy: 0.756     [Train] train_loss: 0.262  train_f1score: 0.885  train_accuracy: 0.886  
Epoch 6: [Val] val_loss: 0.849  val_f1score: 0.688  val_accuracy: 0.693     [Train] train_loss: 0.235  train_f1score: 0.898  train_accuracy: 0.899  
Epoch 7: [Val] val_loss: 0.563  val_f1score: 0.783  val_accuracy: 0.784     [Train] train_loss: 0.205  train_f1score: 0.914  train_accuracy: 0.914  
Epoch 8: [Val] val_loss: 0.877  val_f1score: 0.719  val_accuracy: 0.723     [Train] train_loss: 0.170  train_f1score: 0.929  train_accuracy: 0.929  


Monitored metric val_loss did not improve in the last 5 records. Best score: 0.467. Signaling Trainer to stop.


Epoch 9: [Val] val_loss: 0.954  val_f1score: 0.723  val_accuracy: 0.726     [Train] train_loss: 0.137  train_f1score: 0.945  train_accuracy: 0.945  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt


[Test] test_loss: 0.467 test_f1score: 0.773 test_accuracy: 0.774 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.7744901776313782
      test_f1score           0.773139238357544
        test_loss           0.46712708473205566
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [8]:
# save best_f1_scores_all to ./perturbation_samples/results/
if not os.path.exists('./processed_dataset/perturbation_samples/results/'):
    os.makedirs('./processed_dataset/perturbation_samples/results/')
np.save('./processed_dataset/perturbation_samples/results/best_f1_scores_all.npy', np.array(best_f1_scores_all))

## Feature perturbation analysis

In [ ]:
from torcheeg.datasets import MemoryAgeDataset_perturbation

In [ ]:
perturbation_features = ['ACC_theta', 'CA1_theta', 'BLA_theta', 
                         'ACC_beta', 'CA1_beta', 'BLA_beta', 
                         'ACC_sGamma', 'CA1_sGamma', 'BLA_sGamma', 
                         'ACC_fGamma', 'CA1_fGamma', 'BLA_fGamma']

channel_map = {'ACC': 0, 'CA1': 1, 'BLA': 2}
freq_map = {'theta': [6,12], 'beta': [20,30], 'sGamma': [30,50], 'fGamma': [60,90]}

In [ ]:
test_pertur_f1_all = {}

for erased_feature in perturbation_features:

    test_pertur_f1_all[erased_feature] = []

    print('='*100)
    print(f'                    erased_feature: {erased_feature}')
    print('='*100)

    # seperate the erased_feature into channel and freq
    perturbation_channel = channel_map[erased_feature.split('_')[0]]  # e.g., 'ACC' -> 0, 'CA1' -> 1, 'BLA' -> 2
    perturbation_freq = freq_map[erased_feature.split('_')[1]]        # e.g., 'theta' -> [6,12]

    # perturbate the dataset
    dataset_perturbation = MemoryAgeDataset_perturbation(io_path=f'./processed_dataset/perturbation_samples/{erased_feature}',  
                                                         root_path='./raw_dataset/data_memory_age',
                                                         online_transform=transforms.ToTensor(),
                                                         num_worker=1,
                                                         io_mode='pickle',
                                                         perturbation_channel=perturbation_channel,
                                                         perturbation_freq=perturbation_freq)
    
    k_fold = KFoldCrossSubject(n_splits=12, shuffle=False, split_path=f'./processed_dataset/perturbation_samples/{erased_feature}/cross_subject_k_fold_spilt')
    # shulffle=False, so the order of the subjects in test is ym212, ym213, ..., ym227
    # n_spilts=the num of subjects, so leave-one-subject-out cross validation

    for i, (_, test_dataset) in enumerate(k_fold.split(dataset_perturbation)):
        
        # Initialize the model
        model = LFP_Transformer_LARGE(depth=5)  # 5 transformer blocks, namely, x LARGE MODEL
        
        # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
        # The major parameters of the trainer are defined in ClassifierTrainer
        trainer = ClassifierTrainer(model=model,
                                    lr=1e-4,
                                    devices=1,
                                    accelerator='gpu',
                                    num_classes=2,
                                    weight_decay=1e-4,
                                    save_attention=False,
                                    save_test_f1=True)      
        # we don't need to save the attention scores during the permutation test phase
        # therefore save_attention=False

        # DataLoader for the test dataset
        test_loader = DataLoader(test_dataset,
                                 batch_size=128,
                                 shuffle=False,
                                 num_workers=CPU_num)

        trainer.test(test_loader, fold_idx=i)

        test_pertur_f1_all[erased_feature].append(trainer.test_f1.cpu().numpy())

        print('\n\n\n')


                    erased_feature: ACC_theta
dataset does not exist at path ./processed_dataset/perturbation_samples/ACC_theta, generating files to path...


[PROCESS]: 100%|██████████| 24/24 [04:41<00:00, 11.72s/it]


dataset already exists at path ./processed_dataset/perturbation_samples/ACC_theta, reading from path...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at ./processed_dataset/Model_Checkpoints/Best_model_Fold0.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


-------------------- FOLD 0 -------------------
train_subjects: ['ym226', 'ym214', 'ym224', 'ym220', 'ym213', 'ym227', 'ym215', 'ym222', 'ym218', 'ym223', 'ym219']
test_subjects: ['ym212']
------------------------------------------------------


...

[Test] test_loss: 1.007 test_f1score: 0.573 test_accuracy: 0.618 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6184052228927612
      test_f1score          0.5734939575195312
        test_loss            1.007129192352295
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]






-------------------- FOLD 11 -------------------
train_subjects: ['ym226', 'ym214', 'ym224', 'ym220', 'ym213', 'ym212', 'ym215', 'ym222', 'ym218', 'ym223', 'ym219']
test_subjects: ['ym227']
------------------------------------------------------


Loaded model weights from checkpoint at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt


[Test] test_loss: 0.468 test_f1score: 0.776 test_accuracy: 0.778 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.7776889204978943
      test_f1score          0.7762436866760254
        test_loss           0.4683506488800049
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────






In [ ]:
# save test_pertur_f1_all to ./perturbation_samples/results/
if not os.path.exists('./processed_dataset/perturbation_samples/results/'):
    os.makedirs('./processed_dataset/perturbation_samples/results/')
np.save('./processed_dataset/perturbation_samples/results/test_pertur_f1_all.npy', test_pertur_f1_all)